In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

# ====== Step 1: Dataset dan Transform ======
train_dir = "/workspace/dataset_vit_aug1/dataset_vit_aug/train"

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
labels = train_dataset.targets  # Label untuk StratifiedKFold

print(f"📊 Total Dataset: {len(train_dataset)}")
print(f"📂 Classes: {train_dataset.classes}")

# ====== Step 2: K-Fold Setup ======
n_splits = 5
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "model_vit_tuning.pth"
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
all_reports = []
fold_accuracies = []

# ====== Step 3: Loop K-Fold ======
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\n📚 Fold {fold + 1}")

    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # ====== Step 4: Load Model dan Fine-Tune dari .pth ======
    model = models.vit_b_16(pretrained=True)
    model.heads = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.heads.head.in_features, len(train_dataset.classes))
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scaler = GradScaler()  # AMP scaler

    # ====== Step 5: Training Loop with AMP & tqdm ======
    for epoch in range(5):  # Ubah jumlah epoch jika perlu
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/5", leave=False)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            loop.set_postfix(loss=loss.item())

    # ====== Step 6: Evaluasi per fold ======
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    report = classification_report(y_true, y_pred, output_dict=True, target_names=train_dataset.classes)
    accuracy = accuracy_score(y_true, y_pred)
    fold_accuracies.append({'Fold': f'Fold {fold+1}', 'Accuracy': accuracy})
    all_reports.append(report)

# ====== Step 7: Simpan hasil evaluasi ======
rows = []
for i, report in enumerate(all_reports):
    for cls in train_dataset.classes:
        row = {
            'Fold': f'Fold {i+1}',
            'Class': cls,
            'Precision': report[cls]['precision'],
            'Recall': report[cls]['recall'],
            'F1-Score': report[cls]['f1-score'],
            'Support': report[cls]['support']
        }
        rows.append(row)

df_summary = pd.DataFrame(rows)
df_summary.to_csv("kfold_finetune_vit_evaluation.csv", index=False)

df_acc = pd.DataFrame(fold_accuracies)
df_acc.to_csv("kfold_finetune_vit_accuracy.csv", index=False)

print("\n✅ Hasil evaluasi disimpan ke:")
print(" - kfold_finetune_vit_evaluation.csv (per-class metrics)")
print(" - kfold_finetune_vit_accuracy.csv (accuracy per fold)")


📊 Total Dataset: 150992
📂 Classes: ['A', 'AFIB', 'AFL', 'L', 'N', 'R', 'V']

📚 Fold 1


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_8791/2408246260.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det


📚 Fold 2


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_8791/2408246260.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det


📚 Fold 3


...


📚 Fold 4


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_8791/2408246260.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det


📚 Fold 5


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_8791/2408246260.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det


✅ Hasil evaluasi disimpan ke:
 - kfold_finetune_vit_evaluation.csv (per-class metrics)
 - kfold_finetune_vit_accuracy.csv (accuracy per fold)


In [2]:
pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 70.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 78.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch torchvision torchaudio


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tqdm


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

# ====== Step 1: Dataset dan Transform ======
train_dir = "/workspace/dataset_vit_aug1/dataset_vit_aug/train"
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
labels = train_dataset.targets  # Label untuk StratifiedKFold

print(f"📊 Total Dataset: {len(train_dataset)}")
print(f"📂 Classes: {train_dataset.classes}")

# ====== Step 2: Konfigurasi K-Fold dan Training ======
n_splits = 5
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "model_vit_tuning.pth"
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

all_reports = []
fold_accuracies = []

# ====== Step 3: Fungsi Hitung Sensitivity dan Specificity ======
def calculate_specificity_sensitivity(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    metrics = {}
    for i, label in enumerate(labels):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        sensitivity = TP / (TP + FN) if TP + FN != 0 else 0
        specificity = TN / (TN + FP) if TN + FP != 0 else 0
        metrics[label] = {'sensitivity': sensitivity, 'specificity': specificity}
    return metrics

# ====== Step 4: Proses K-Fold Cross Validation ======
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\n📚 Fold {fold + 1}")

    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Load dan sesuaikan model
    model = models.vit_b_16(pretrained=True)
    model.heads = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.heads.head.in_features, len(train_dataset.classes))
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scaler = GradScaler()

    # ====== Step 5: Proses Training per Fold ======
    for epoch in range(10):
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/10", leave=False)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            loop.set_postfix(loss=loss.item())

    # ====== Step 6: Evaluasi Model ======
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    report = classification_report(y_true, y_pred, output_dict=True, target_names=train_dataset.classes)
    accuracy = accuracy_score(y_true, y_pred)
    sensi_speci = calculate_specificity_sensitivity(y_true, y_pred, train_dataset.classes)

    fold_result = {
        'fold': fold + 1,
        'accuracy': accuracy,
        'report': report,
        'sensi_speci': sensi_speci
    }
    all_reports.append(fold_result)
    fold_accuracies.append({'Fold': f'Fold {fold+1}', 'Accuracy': accuracy})

# ====== Step 7: Simpan Evaluasi Detail Per Fold ======
rows = []
for fold_result in all_reports:
    fold_number = fold_result['fold']
    accuracy = fold_result['accuracy']
    report = fold_result['report']
    sensi_speci = fold_result['sensi_speci']

    for cls in train_dataset.classes:
        precision = report[cls]['precision']
        recall = report[cls]['recall']
        f1 = report[cls]['f1-score']
        support = report[cls]['support']
        sensitivity = sensi_speci[cls]['sensitivity']
        specificity = sensi_speci[cls]['specificity']

        row = {
            'Fold': f"Fold {fold_number}",
            'Class': cls,
            'Accuracy': round(accuracy, 4),
            'Precision': round(precision, 4),
            'Recall': round(recall, 4),
            'F1-Score': round(f1, 4),
            'Support': support,
            'Sensitivity': round(sensitivity, 4),
            'Specificity': round(specificity, 4)
        }
        rows.append(row)

df_summary = pd.DataFrame(rows)
df_summary.to_csv("kfold_finetune_vit_evaluation.csv", index=False)

# ====== Step 8: Identifikasi Fold Terbaik & Simpan Tabel Detail ======
best_fold = max(all_reports, key=lambda x: x['accuracy'])
print(f"\n🏆 Fold terbaik: Fold {best_fold['fold']} dengan akurasi {best_fold['accuracy']:.4f}")

best_rows = []
for cls in train_dataset.classes:
    precision = best_fold['report'][cls]['precision']
    recall = best_fold['report'][cls]['recall']
    f1 = best_fold['report'][cls]['f1-score']
    sensitivity = best_fold['sensi_speci'][cls]['sensitivity']
    specificity = best_fold['sensi_speci'][cls]['specificity']

    best_rows.append({
        'Class': cls,
        'Precision': round(precision, 4),
        'Sensitivity': round(sensitivity, 4),
        'Specificity': round(specificity, 4),
        'F1-Score': round(f1, 4)
    })

df_best = pd.DataFrame(best_rows)
df_best.to_csv("best_fold_class_metrics.csv", index=False)
print("✅ Metrik per kelas dari fold terbaik disimpan ke: best_fold_class_metrics.csv")


📊 Total Dataset: 150992
📂 Classes: ['A', 'AFIB', 'AFL', 'L', 'N', 'R', 'V']

📚 Fold 1


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100.0%
/tmp/ipykernel_390/3650007387.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data whic


📚 Fold 2


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_390/3650007387.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta


📚 Fold 3


...


📚 Fold 4


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_390/3650007387.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta


📚 Fold 5


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_390/3650007387.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta


🏆 Fold terbaik: Fold 4 dengan akurasi 0.9925
✅ Metrik per kelas dari fold terbaik disimpan ke: best_fold_class_metrics.csv


In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

# ====== Step 1: Dataset dan Transform ======
train_dir = "/workspace/dataset_vit_aug1/dataset_vit_aug/train"
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
labels = train_dataset.targets  # Label untuk StratifiedKFold

print(f"📊 Total Dataset: {len(train_dataset)}")
print(f"📂 Classes: {train_dataset.classes}")

# ====== Step 2: Konfigurasi K-Fold dan Training ======
n_splits = 5
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "model_vit_tuning.pth"
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

all_reports = []
fold_accuracies = []

# ====== Step 3: Fungsi Hitung Sensitivity dan Specificity ======
def calculate_specificity_sensitivity(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    metrics = {}
    for i, label in enumerate(labels):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        sensitivity = TP / (TP + FN) if TP + FN != 0 else 0
        specificity = TN / (TN + FP) if TN + FP != 0 else 0
        metrics[label] = {'sensitivity': sensitivity, 'specificity': specificity}
    return metrics

# ====== Step 4: Proses K-Fold Cross Validation ======
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\n📚 Fold {fold + 1}")

    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Load dan sesuaikan model
    model = models.vit_b_16(pretrained=True)
    model.heads = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.heads.head.in_features, len(train_dataset.classes))
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scaler = GradScaler()

    # ====== Step 5: Proses Training per Fold ======
    for epoch in range(10):
        model.train()
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/10", leave=False)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            loop.set_postfix(loss=loss.item())

    # ====== Step 6: Evaluasi Model ======
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    report = classification_report(y_true, y_pred, output_dict=True, target_names=train_dataset.classes)
    accuracy = accuracy_score(y_true, y_pred)
    sensi_speci = calculate_specificity_sensitivity(y_true, y_pred, train_dataset.classes)

    fold_result = {
        'fold': fold + 1,
        'accuracy': accuracy,
        'report': report,
        'sensi_speci': sensi_speci
    }
    all_reports.append(fold_result)
    fold_accuracies.append({'Fold': f'Fold {fold+1}', 'Accuracy': accuracy})

# ====== Step 7: Simpan Evaluasi Detail Per Fold ======
rows = []
for fold_result in all_reports:
    fold_number = fold_result['fold']
    accuracy = fold_result['accuracy']
    report = fold_result['report']
    sensi_speci = fold_result['sensi_speci']

    for cls in train_dataset.classes:
        precision = report[cls]['precision']
        recall = report[cls]['recall']
        f1 = report[cls]['f1-score']
        support = report[cls]['support']
        sensitivity = sensi_speci[cls]['sensitivity']
        specificity = sensi_speci[cls]['specificity']

        row = {
            'Fold': f"Fold {fold_number}",
            'Class': cls,
            'Accuracy': round(accuracy, 4),
            'Precision': round(precision, 4),
            'Recall': round(recall, 4),
            'F1-Score': round(f1, 4),
            'Support': support,
            'Sensitivity': round(sensitivity, 4),
            'Specificity': round(specificity, 4)
        }
        rows.append(row)

df_summary = pd.DataFrame(rows)
df_summary.to_csv("kfold_finetune_vit_evaluation1.csv", index=False)

# ====== Step 7B: Simpan Ringkasan Evaluasi Per Fold (tanpa per kelas) ======
summary_per_fold = []

for fold_result in all_reports:
    fold_number = fold_result['fold']
    accuracy = fold_result['accuracy']
    macro = fold_result['report']['macro avg']

    row = {
        'Fold': f'Fold {fold_number}',
        'Accuracy': round(accuracy, 4),
        'Macro Precision': round(macro['precision'], 4),
        'Macro Recall': round(macro['recall'], 4),
        'Macro F1-Score': round(macro['f1-score'], 4)
    }
    summary_per_fold.append(row)

df_fold_summary = pd.DataFrame(summary_per_fold)
df_fold_summary.to_csv("summary_per_fold.csv", index=False)
print("✅ Ringkasan per fold disimpan ke: summary_per_fold.csv")
# ====== Step 7C: Ringkasan Lengkap Evaluasi per Fold (Full Metrics) ======
summary_table = []

for fold_result in all_reports:
    fold_number = fold_result['fold']
    accuracy = fold_result['accuracy']
    report = fold_result['report']
    sensi_speci = fold_result['sensi_speci']

    precision_vals = [report[cls]['precision'] for cls in train_dataset.classes]
    recall_vals = [report[cls]['recall'] for cls in train_dataset.classes]
    f1_vals = [report[cls]['f1-score'] for cls in train_dataset.classes]
    sensitivities = [v['sensitivity'] for v in sensi_speci.values()]
    specificities = [v['specificity'] for v in sensi_speci.values()]

    row = {
        'Fold': f'Fold {fold_number}',
        'Accuracy': round(accuracy, 6),
        'Precision': round(np.mean(precision_vals), 6),
        'Sensitivity': round(np.mean(sensitivities), 6),
        'Specificity': round(np.mean(specificities), 6),
        'F1-Score': round(np.mean(f1_vals), 6)
    }
    summary_table.append(row)

df_full_summary = pd.DataFrame(summary_table)
df_full_summary.to_csv("summary_fold_all_metrics.csv", index=False)
print("✅ Tabel evaluasi lengkap disimpan ke: summary_fold_all_metrics.csv")

# ====== Step 8: Identifikasi Fold Terbaik & Simpan Tabel Detail ======
best_fold = max(all_reports, key=lambda x: x['accuracy'])
print(f"\n🏆 Fold terbaik: Fold {best_fold['fold']} dengan akurasi {best_fold['accuracy']:.4f}")

best_rows = []
for cls in train_dataset.classes:
    precision = best_fold['report'][cls]['precision']
    recall = best_fold['report'][cls]['recall']
    f1 = best_fold['report'][cls]['f1-score']
    sensitivity = best_fold['sensi_speci'][cls]['sensitivity']
    specificity = best_fold['sensi_speci'][cls]['specificity']

    best_rows.append({
        'Class': cls,
        'Precision': round(precision, 4),
        'Sensitivity': round(sensitivity, 4),
        'Specificity': round(specificity, 4),
        'F1-Score': round(f1, 4)
    })

df_best = pd.DataFrame(best_rows)
df_best.to_csv("best_fold_class_metrics_ViT.csv", index=False)
print("✅ Metrik per kelas dari fold terbaik disimpan ke: best_fold_class_metrics.csv")


📊 Total Dataset: 150992
📂 Classes: ['A', 'AFIB', 'AFL', 'L', 'N', 'R', 'V']

📚 Fold 1


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_619/2431816455.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta


📚 Fold 2


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_619/2431816455.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta


📚 Fold 3


...


📚 Fold 4


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_619/2431816455.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta


📚 Fold 5


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_619/2431816455.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more deta

✅ Ringkasan per fold disimpan ke: summary_per_fold.csv
✅ Tabel evaluasi lengkap disimpan ke: summary_fold_all_metrics.csv

🏆 Fold terbaik: Fold 2 dengan akurasi 0.9932
✅ Metrik per kelas dari fold terbaik disimpan ke: best_fold_class_metrics.csv


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# ====== Device Setup ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📟 Device: {device}")

# ====== Load Dataset dari test_dir ======
test_dir = "dataset_vit/test"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
dataset = datasets.ImageFolder(test_dir, transform=transform)
labels = dataset.targets
classes = dataset.classes
print(f"🧪 Total Data: {len(dataset)} samples")
print(f"📂 Classes: {classes}")

# ====== K-Fold Config ======
n_splits = 5
batch_size = 32
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# ====== Fungsi Hitung Sensitivity & Specificity ======
def calculate_specificity_sensitivity(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    metrics = {}
    for i, label in enumerate(labels):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        sensitivity = TP / (TP + FN) if TP + FN != 0 else 0
        specificity = TN / (TN + FP) if TN + FP != 0 else 0
        metrics[label] = {'sensitivity': sensitivity, 'specificity': specificity}
    return metrics

# ====== Step 1: Evaluasi K-Fold ======
all_reports = []

for fold, (_, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\n📚 Fold {fold + 1}")
    val_subset = Subset(dataset, val_idx)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Load Model
    model = models.vit_b_16(pretrained=True)
    model.heads = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.heads.head.in_features, len(classes))
    )
    model.load_state_dict(torch.load("model_vit_tuning.pth", map_location=device))
    model = model.to(device)
    model.eval()

    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels_batch in tqdm(val_loader, desc=f"Evaluating Fold {fold+1}"):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels_batch.numpy())

    report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)
    accuracy = accuracy_score(y_true, y_pred)
    sensi_speci = calculate_specificity_sensitivity(y_true, y_pred, classes)

    all_reports.append({
        'fold': fold + 1,
        'accuracy': accuracy,
        'report': report,
        'sensi_speci': sensi_speci,
        'y_true': y_true,
        'y_pred': y_pred
    })

# ====== Step 2: Simpan Detail Evaluasi Tiap Fold ======
rows = []
for fold_result in all_reports:
    fold = fold_result['fold']
    report = fold_result['report']
    sensi_speci = fold_result['sensi_speci']
    acc = fold_result['accuracy']
    for cls in classes:
        rows.append({
            'Fold': f"Fold {fold}",
            'Class': cls,
            'Accuracy': round(acc, 4),
            'Precision': round(report[cls]['precision'], 4),
            'Recall': round(report[cls]['recall'], 4),
            'F1-Score': round(report[cls]['f1-score'], 4),
            'Sensitivity': round(sensi_speci[cls]['sensitivity'], 4),
            'Specificity': round(sensi_speci[cls]['specificity'], 4),
            'Support': report[cls]['support']
        })
df_detail = pd.DataFrame(rows)
os.makedirs("results_eval_kfold_vit", exist_ok=True)
df_detail.to_csv("results_eval_kfold_vit/kfold_detail_vit_tuned.csv", index=False)

# ====== Step 3: Simpan Ringkasan per Fold ======
summary_per_fold = []
for result in all_reports:
    macro = result['report']['macro avg']
    summary_per_fold.append({
        'Fold': f"Fold {result['fold']}",
        'Accuracy': round(result['accuracy'], 4),
        'Macro Precision': round(macro['precision'], 4),
        'Macro Recall': round(macro['recall'], 4),
        'Macro F1-Score': round(macro['f1-score'], 4)
    })
df_summary = pd.DataFrame(summary_per_fold)
df_summary.to_csv("results_eval_kfold_vit/kfold_summary_vit_tuned.csv", index=False)

# ====== Step 4: Ringkasan Lengkap Rata-rata Metrik ======
summary_table = []
for result in all_reports:
    fold = result['fold']
    rep = result['report']
    sensi_speci = result['sensi_speci']
    precision_vals = [rep[cls]['precision'] for cls in classes]
    recall_vals = [rep[cls]['recall'] for cls in classes]
    f1_vals = [rep[cls]['f1-score'] for cls in classes]
    sensitivities = [v['sensitivity'] for v in sensi_speci.values()]
    specificities = [v['specificity'] for v in sensi_speci.values()]
    summary_table.append({
        'Fold': f"Fold {fold}",
        'Accuracy': round(result['accuracy'], 6),
        'Precision': round(np.mean(precision_vals), 6),
        'Sensitivity': round(np.mean(sensitivities), 6),
        'Specificity': round(np.mean(specificities), 6),
        'F1-Score': round(np.mean(f1_vals), 6)
    })
df_all_summary = pd.DataFrame(summary_table)
df_all_summary.to_csv("results_eval_kfold_vit/kfold_full_summary_vit_tuned.csv", index=False)

# ====== Step 5: Fold Terbaik + Confusion Matrix ======
best_fold = max(all_reports, key=lambda x: x['accuracy'])
print(f"\n🏆 Fold terbaik: Fold {best_fold['fold']} dengan akurasi {best_fold['accuracy']:.4f}")

# Simpan Metrik Per Kelas dari Fold Terbaik
best_rows = []
for cls in classes:
    best_rows.append({
        'Class': cls,
        'Precision': round(best_fold['report'][cls]['precision'], 4),
        'Sensitivity': round(best_fold['sensi_speci'][cls]['sensitivity'], 4),
        'Specificity': round(best_fold['sensi_speci'][cls]['specificity'], 4),
        'F1-Score': round(best_fold['report'][cls]['f1-score'], 4)
    })
df_best = pd.DataFrame(best_rows)
df_best.to_csv("results_eval_kfold_vit/best_fold_class_metrics_vit.csv", index=False)

# Plot Confusion Matrix Fold Terbaik
cm = confusion_matrix(best_fold['y_true'], best_fold['y_pred'])
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted")
plt.ylabel("True Label")
plt.title(f"Confusion Matrix - Fold {best_fold['fold']}")
plt.tight_layout()
plt.savefig(f"results_eval_kfold_vit/confusion_matrix_fold_{best_fold['fold']}.png")
plt.close()
print(f"🖼️ Confusion matrix disimpan ke: results_eval_kfold_vit/confusion_matrix_fold_{best_fold['fold']}.png")


📟 Device: cuda
🧪 Total Data: 33335 samples
📂 Classes: ['A', 'AFIB', 'AFL', 'L', 'N', 'R', 'V']

📚 Fold 1


c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Evaluating Fold 1: 100%|██████████| 209/209 [01:53<00:00,  1.83it/s]
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warn


📚 Fold 2


Evaluating Fold 2: 100%|██████████| 209/209 [01:58<00:00,  1.77it/s]
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



📚 Fold 3


...


📚 Fold 4


Evaluating Fold 4: 100%|██████████| 209/209 [01:59<00:00,  1.74it/s]
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



📚 Fold 5


Evaluating Fold 5: 100%|██████████| 209/209 [01:59<00:00,  1.74it/s]



🏆 Fold terbaik: Fold 1 dengan akurasi 0.9924
🖼️ Confusion matrix disimpan ke: results_eval_kfold_vit/confusion_matrix_fold_1.png


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# ====== Device Setup ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📟 Device: {device}")

# ====== Load Dataset dari train_dir ======
train_dir = "dataset_vit_aug/train"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])
dataset = datasets.ImageFolder(train_dir, transform=transform)
labels = dataset.targets
classes = dataset.classes
print(f"📊 Total Train Data: {len(dataset)} samples")
print(f"📂 Classes: {classes}")

# ====== K-Fold Config ======
n_splits = 5
batch_size = 32
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# ====== Fungsi Hitung Sensitivity & Specificity ======
def calculate_specificity_sensitivity(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    metrics = {}
    for i, label in enumerate(labels):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)
        sensitivity = TP / (TP + FN) if TP + FN != 0 else 0
        specificity = TN / (TN + FP) if TN + FP != 0 else 0
        metrics[label] = {'sensitivity': sensitivity, 'specificity': specificity}
    return metrics

# ====== Step 1: Evaluasi K-Fold ======
all_reports = []

for fold, (_, val_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\n📚 Fold {fold + 1}")
    val_subset = Subset(dataset, val_idx)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Load Model
    model = models.vit_b_16(pretrained=True)
    model.heads = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.heads.head.in_features, len(classes))
    )
    model.load_state_dict(torch.load("model_vit_tuning.pth", map_location=device))
    model = model.to(device)
    model.eval()

    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels_batch in tqdm(val_loader, desc=f"Evaluating Fold {fold+1}"):
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(labels_batch.numpy())

    report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)
    accuracy = accuracy_score(y_true, y_pred)
    sensi_speci = calculate_specificity_sensitivity(y_true, y_pred, classes)

    all_reports.append({
        'fold': fold + 1,
        'accuracy': accuracy,
        'report': report,
        'sensi_speci': sensi_speci,
        'y_true': y_true,
        'y_pred': y_pred
    })

# ====== Step 2: Simpan Detail Evaluasi Tiap Fold ======
rows = []
for fold_result in all_reports:
    fold = fold_result['fold']
    report = fold_result['report']
    sensi_speci = fold_result['sensi_speci']
    acc = fold_result['accuracy']
    for cls in classes:
        rows.append({
            'Fold': f"Fold {fold}",
            'Class': cls,
            'Accuracy': round(acc, 4),
            'Precision': round(report[cls]['precision'], 4),
            'Recall': round(report[cls]['recall'], 4),
            'F1-Score': round(report[cls]['f1-score'], 4),
            'Sensitivity': round(sensi_speci[cls]['sensitivity'], 4),
            'Specificity': round(sensi_speci[cls]['specificity'], 4),
            'Support': report[cls]['support']
        })
df_detail = pd.DataFrame(rows)
os.makedirs("results_eval_kfold_vit_train", exist_ok=True)
df_detail.to_csv("results_eval_kfold_vit_train/kfold_detail_vit_tuned_train.csv", index=False)

# ====== Step 3: Simpan Ringkasan per Fold ======
summary_per_fold = []
for result in all_reports:
    macro = result['report']['macro avg']
    summary_per_fold.append({
        'Fold': f"Fold {result['fold']}",
        'Accuracy': round(result['accuracy'], 4),
        'Macro Precision': round(macro['precision'], 4),
        'Macro Recall': round(macro['recall'], 4),
        'Macro F1-Score': round(macro['f1-score'], 4)
    })
df_summary = pd.DataFrame(summary_per_fold)
df_summary.to_csv("results_eval_kfold_vit_train/kfold_summary_vit_tuned_train.csv", index=False)

# ====== Step 4: Ringkasan Lengkap Rata-rata Metrik ======
summary_table = []
for result in all_reports:
    fold = result['fold']
    rep = result['report']
    sensi_speci = result['sensi_speci']
    precision_vals = [rep[cls]['precision'] for cls in classes]
    recall_vals = [rep[cls]['recall'] for cls in classes]
    f1_vals = [rep[cls]['f1-score'] for cls in classes]
    sensitivities = [v['sensitivity'] for v in sensi_speci.values()]
    specificities = [v['specificity'] for v in sensi_speci.values()]
    summary_table.append({
        'Fold': f"Fold {fold}",
        'Accuracy': round(result['accuracy'], 6),
        'Precision': round(np.mean(precision_vals), 6),
        'Sensitivity': round(np.mean(sensitivities), 6),
        'Specificity': round(np.mean(specificities), 6),
        'F1-Score': round(np.mean(f1_vals), 6)
    })
df_all_summary = pd.DataFrame(summary_table)
df_all_summary.to_csv("results_eval_kfold_vit_train/kfold_full_summary_vit_tuned_train.csv", index=False)

# ====== Step 5: Fold Terbaik + Confusion Matrix ======
best_fold = max(all_reports, key=lambda x: x['accuracy'])
print(f"\n🏆 Fold terbaik: Fold {best_fold['fold']} dengan akurasi {best_fold['accuracy']:.4f}")

# Simpan Metrik Per Kelas dari Fold Terbaik
best_rows = []
for cls in classes:
    best_rows.append({
        'Class': cls,
        'Precision': round(best_fold['report'][cls]['precision'], 4),
        'Sensitivity': round(best_fold['sensi_speci'][cls]['sensitivity'], 4),
        'Specificity': round(best_fold['sensi_speci'][cls]['specificity'], 4),
        'F1-Score': round(best_fold['report'][cls]['f1-score'], 4)
    })
df_best = pd.DataFrame(best_rows)
df_best.to_csv("results_eval_kfold_vit_train/best_fold_class_metrics_vit_train.csv", index=False)

# Plot Confusion Matrix Fold Terbaik
cm = confusion_matrix(best_fold['y_true'], best_fold['y_pred'])
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel("Predicted")
plt.ylabel("True Label")
plt.title(f"Confusion Matrix - Fold {best_fold['fold']} (Train Data)")
plt.tight_layout()
plt.savefig(f"results_eval_kfold_vit_train/confusion_matrix_fold_{best_fold['fold']}_train.png")
plt.close()
print(f"🖼️ Confusion matrix disimpan ke: results_eval_kfold_vit_train/confusion_matrix_fold_{best_fold['fold']}_train.png")


📟 Device: cuda
📊 Total Train Data: 150992 samples
📂 Classes: ['A', 'AFIB', 'AFL', 'L', 'N', 'R', 'V']

📚 Fold 1


c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Evaluating Fold 1: 100%|██████████| 944/944 [09:09<00:00,  1.72it/s]
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warn


📚 Fold 2


Evaluating Fold 2: 100%|██████████| 944/944 [09:04<00:00,  1.73it/s]
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



📚 Fold 3


...


📚 Fold 4


Evaluating Fold 4: 100%|██████████| 944/944 [08:30<00:00,  1.85it/s]
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



📚 Fold 5


Evaluating Fold 5: 100%|██████████| 944/944 [08:28<00:00,  1.86it/s]



🏆 Fold terbaik: Fold 4 dengan akurasi 0.9903
🖼️ Confusion matrix disimpan ke: results_eval_kfold_vit_train/confusion_matrix_fold_4_train.png
